Để bắt đầu, chúng ta cần load thư viện tidyr.

In [ ]:
# The easiest way to get tidyr is to install the whole tidyverse:
# install.packages("tidyr")
# install.packages("dplyr")
library(tidyr)
library(dplyr)

Tác giả của tidyr, Hadley Wickham, có thảo luận về triết lý đằng sau việc dọn dẹp dữ liệu (tidy data) trong bài báo: http://vita.had.co.nz/papers/tidy-data.pdf. Bài báo này nên đọc nếu bạn làm việc với phân tích dữ liệu.

Tidy data là định dạng chuẩn cho các phân tích exploration và bước tiền xử lý cho các công cụ khác làm việc mượt mà hơn trên dữ liệu đã được dọn dẹp. Cụ thể, tidy data thỏa mãn ba điều kiện.
<ul>
	<li>Mỗi thuộc tính chỉ thuộc một cột dữ liệu</li>
	<li>Mỗi đối tượng chỉ chứa trên một dòng dữ liệu</li>
	<li>Mỗi loại đối tượng chỉ thuộc một bảng dữ liệu</li>
</ul>

<h2>Mỗi thuộc tính chỉ thuộc một cột dữ liệu</h2>
Những tập dữ liệu không thỏa các điều kiện trên được xem là dữ liệu hỗn độn (messy data). Vấn đề đầu tiên đó là cột dữ liệu chỉ chứa giá trị chứ không có tên thuộc tính. Ta lấy tập dữ liệu 'students' sau làm ví dụ cho trường hợp này.

In [2]:
# Initialize
# number of letter grades (A - E)
.ng <- 5
# max number of students/group
.gmax <- 8

### column headers are values, not variable names

set.seed(1234)
students <- data.frame(
  grade = LETTERS[1:.ng],
  male = sample(0:.gmax, .ng, replace = TRUE),
  female = sample(0:.gmax, .ng, replace = TRUE)
)

### multiple variables are stored in one column

set.seed(1211)
students2 <- data.frame(
  grade = LETTERS[1:.ng],
  male_1 = sample(0:.gmax, .ng, replace = TRUE),
  female_1 = sample(0:.gmax, .ng, replace = TRUE),
  male_2 = sample(0:.gmax, .ng, replace = TRUE),
  female_2 = sample(0:.gmax, .ng, replace = TRUE)
)

### loading data
students3 <- read.csv("data/students3.csv", na.strings = "", stringsAsFactors = FALSE)
students4 <- read.csv("data/students4.csv", stringsAsFactors = FALSE)
sat <- read.csv("data/sat13.csv", stringsAsFactors = FALSE)

### a single observational unit is stored in multiple tables

passed <- students4 %>%
  select(name, class, final) %>%
  filter(final == "A" | final == "B")

failed <- students4 %>%
  select(name, class, final) %>%
  filter(final == "C" | final == "D" | final == "E")

In [3]:
students

grade,male,female
A,1,5
B,5,0
C,5,2
D,5,5
E,7,4


Cột đầu tiên biểu diễn xếp hạng có thể có của từng học sinh. Cột thứ hai và ba thể hiện số lượng nữ sinh và nam sinh tương ứng với xếp hạng này. Tập dữ liệu này thật ra có ba thuộc tính: grade, sex, và count. Thuộc tính đầu tiên là grade đã thỏa điều kiện với cột đầu tiên. Thuộc tính thứ hai sex được biểu diễn bởi hai cột dữ liệu hai và ba. Thuộc tính thứ ba count là số lượng học sinh cho mỗi kết hợp của grade và sex.

Đễ dọn dẹp tập dữ liệu students, ta cần một cột dữ liệu cho từng thuộc tính trên. Chúng ta sẽ dùng hàm gather() từ thư viện tidyr để hoàn thành tác vụ này.

In [4]:
args(gather)

function (data, key = "key", value = "value", ..., na.rm = FALSE, 
    convert = FALSE, factor_key = FALSE) 
NULL

In [5]:
# Gather takes multiple columns and collapses into key-value pairs,
# duplicating all other columns as needed. You use ‘gather()’ when
# you notice that you have columns that are not variables.
# -y: exclude column
gather(data=students, key="sex", value="count", -"grade")

grade,sex,count
A,male,1
B,male,5
C,male,5
D,male,5
E,male,7
A,female,5
B,female,0
C,female,2
D,female,5
E,female,4


Mỗi dòng dữ liệu bây giờ chỉ thể hiện một đối tượng duy nhất được đặc trưng bởi kết hợp giữa thuộc tính grade và sex. Mỗi thuộc tính (grade, sex, và count) được lưu bởi duy nhất một cột dữ liệu. Ta đã có tidy data.

Điều quan trọng chúng ta cần hiểu được ý nghĩa của các đối số truyền vào hàm gather(). Đối số data ở đây là students. Đối số key và value gồm có sex và count để đặt tên cho cột thuộc tính của tidy dataset. Đối số sau cùng là grade, ta muốn gom tất cả các cột dữ liệu ngoại trừ cột dữ liệu grade (vì grade đã thảo điều kiện tidy data).

Tập dữ liệu lộn xộn thứ hai ta sẽ quan sát có nhiều thuộc tính được lưu bên trong duy nhất một cột thuộc tính. Ta có tập dữ liệu student2 như sau:

In [6]:
students2

grade,male_1,female_1,male_2,female_2
A,3,4,3,4
B,6,4,3,5
C,7,4,3,8
D,4,0,8,1
E,1,1,2,7


Tập dữ liệu này khá tương đồng với tập dữ liệu ban đầu, ngoại trừ ta có thêm hai lớp phân biệt 1 và 2. Chúng ta có thêm tổng số lượng cho từng lớp. students2 bị xếp vào tập dữ liệu lộn xộn (messy data) khi cột thuộc tính là các giá trị (male_1, female_1,...) mà không phải là tên thuộc tính (sex, class, và count).

Tuy nhiên, nó cũng chứa nhiều thuộc tính được lưu trong mỗi cột (sex và class) là dấu hiệu khác của dữ liệu lộn xộn. Dọn dẹp tập dữ liệu này gồm hai bước. Ta hãy dùng hàm gather() để nạp chồng các cột dữ liệu của students2 như lúc nãy. Lần này, đặt tên cho đối số 'key' là sex_class và 'value' là count. Ta lưu kết quả trả về vào biến res.

In [7]:
res <- gather(students2, sex_class, count, -grade)
head(res)

grade,sex_class,count
A,male_1,3
B,male_1,6
C,male_1,7
D,male_1,4
E,male_1,1
A,female_1,4


Ta đã hoàn thành nửa bước dọn dẹp dữ liệu, nhưng chúng ta vẫn còn hai thuộc tính sex và class được lưu trong cùng một cột sex_class. tidyr cung cấp hàm separate() hữu dụng cho việc chia một cột dữ liệu thành nhiều cột. Ta gọi hàm này cho res để chia cột sex_class thành sex và class.

In [8]:
args(separate)

function (data, col, into, sep = "[^[:alnum:]]+", remove = TRUE, 
    convert = FALSE, extra = "warn", fill = "warn", ...) 
NULL

In [9]:
head(separate(res, sex_class, c("sex", "class")))

grade,sex,class,count
A,male,1,3
B,male,1,6
C,male,1,7
D,male,1,4
E,male,1,1
A,female,1,4


Thật tiện lợi, hàm separate() có khả năng biết được cách để chia cột sex\_class. Mặc định hàm này chia dựa trên dấu gạch dưới "_", ta có thể chỉ định kí tự phân chia bằng đối số 'sep'.

Dọn dẹp tập dữ liệu students2 cần hai hàm gather() và separate(). Như vậy, ta có thể sử dụng kĩ thuật chaining bên dplyr bằng toán tử %&gt;% để kết nhiều hàm cùng lúc.

In [10]:
students2 %>%
gather(sex_class, count, -grade) %>%
separate(sex_class, c("sex", "class")) %>%
head

grade,sex,class,count
A,male,1,3
B,male,1,6
C,male,1,7
D,male,1,4
E,male,1,1
A,female,1,4


<h2>Mỗi đối tượng chỉ chứa trên một dòng dữ liệu</h2>
Dấu hiệu thứ ba của tập dữ liệu hỗn độn là khi các thuộc tính được lưu trên cả dòng và cột. Ta lấy ví dụ ở tập dữ liệu students3 sau:

In [11]:
head(students3)

name,test,class1,class2,class3,class4,class5
Sally,midterm,A,NA,B,NA,NA
Sally,final,C,NA,C,NA,NA
Jeff,midterm,NA,D,NA,A,NA
Jeff,final,NA,E,NA,C,NA
Roger,midterm,NA,C,NA,NA,B
Roger,final,NA,A,NA,NA,A


Trong students3, ta có giá trị xếp hạng midterm và final exam cho năm học sinh, mỗi học sinh đăng ký học năm lớp tương ứng.

Thuộc tính đầu tiên, name, ta sẽ giữ nguyên. Năm cột dữ liệu còn lại từ class1 đến class5 nên được gom thành cột thuộc tính class. Giá trị trong cột test là midterm và final cũng nên được gom thành các cột thuộc tính tương ứng.

In [12]:
students3 %>%
gather(class, grade, class1:class5, na.rm = TRUE) %>%
head

,name,test,class,grade
1,Sally,midterm,class1,A
2,Sally,final,class1,C
9,Brian,midterm,class1,B
10,Brian,final,class1,B
13,Jeff,midterm,class2,D
14,Jeff,final,class2,E


Bước tiếp theo ta sẽ cần đến hàm spread().

In [13]:
args(spread)

function (data, key, value, fill = NA, convert = FALSE, drop = TRUE, 
    sep = NULL) 
NULL

In [14]:
# Spread a key-value pair across multiple columns.
# spread(data, key, value, fill = NA, convert = FALSE, drop = TRUE)
students3 %>%
gather(class, grade, class1:class5, na.rm = TRUE) %>%
spread(test, grade) %>%
head

name,class,final,midterm
Brian,class1,B,B
Brian,class5,C,A
Jeff,class2,E,D
Jeff,class4,C,A
Karen,class3,C,C
Karen,class4,A,A


Sau cùng, chúng ta muốn các giá trị trong cột class đơn giản là 1, 2, ..., 5 thay vì class1, class2, ..., class5. Chúng ta có thể dùng hàm extract_numeric() của tidyr để làm việc này.

In [15]:
args(extract_numeric)

function (x) 
NULL

In [16]:
students3 %>%
gather(class, grade, class1:class5, na.rm = TRUE) %>%
spread(test, grade) %>%
mutate(class = extract_numeric(class)) %>%
head

extract_numeric() is deprecated: please use readr::parse_number() instead


name,class,final,midterm
Brian,1,B,B
Brian,5,C,A
Jeff,2,E,D
Jeff,4,C,A
Karen,3,C,C
Karen,4,A,A


<h2>Mỗi loại đối tượng chỉ thuộc một bảng dữ liệu</h2>
Trường hợp thứ tư của dữ liệu hỗn độn đó là nhiều loại đối tượng khác nhau được lưu chung một bảng. Ta xét ví dụ này với students4


In [17]:
head(students4)

id,name,sex,class,midterm,final
168,Brian,F,1,B,B
168,Brian,F,5,A,C
588,Sally,M,1,A,C
588,Sally,M,3,B,C
710,Jeff,M,2,D,E
710,Jeff,M,4,A,C


students4 khá giống students3. Điểm khác biệt duy nhất đó là students4 có thêm cột id cho mỗi học sinh cùng với giới tính (M = male; F = female).

Nhìn sơ qua, ta không thấy gì bất ổn với students4. Tất cả các cột đều là thuộc tính và tất cả các đối tượng đều thuộc một dòng. Tuy nhiên, để ý cột id, name, và sex lặp lại hai lần, điều này có vẻ dư thừa. Đây là dấu hiệu dữ liệu của chúng ta chứa nhiều loại đối tượng trên cùng một bảng dữ liệu.

Giải pháp của chúng ta chia students4 thành hai bảng tách biệt -- một bảng chứa thông tin cơ bản của student (id, name, và sex) bảng còn lại chứa grades (id, class, midterm, final).

In [18]:
# bảng dữ liệu chứa thông tin học sinh
student_info <- students4 %>%
select(id, name, sex) %>%
unique

head(student_info)

,id,name,sex
1,168,Brian,F
3,588,Sally,M
5,710,Jeff,M
7,731,Roger,F
9,908,Karen,M


In [19]:
gradebook <- students4 %>% select(id, class, midterm, final)
head(gradebook)

id,class,midterm,final
168,1,B,B
168,5,A,C
588,1,A,C
588,3,B,C
710,2,D,E
710,4,A,C


Điều quan trọng cần lưu ý là chúng ta đặt cột id ở cả hai bảng dữ liệu. id ở đây là 'primary key' để tham chiếu đến thông tin học sinh. Không có chỉ định cột id cho bảng gradebook, ta không thể xác định được đây là xếp hạng của học sinh nào.

Trường hợp cuối cùng của dữ liệu hỗn độn đó là một loại đối tượng lại được lưu ở nhiều bảng khác nhau. Trái ngược với trường hợp thứ tư. Ta lấy ví dụ sau:

In [20]:
# danh sách đậu
passed

name,class,final
Brian,1,B
Roger,2,A
Roger,5,A
Karen,4,A


In [21]:
# danh sách rớt
failed

name,class,final
Brian,5,C
Sally,1,C
Sally,3,C
Jeff,2,E
Jeff,4,C
Karen,3,C


Các giáo viên quyết định xem xếp hạng final như thế nào để quyết định học sinh đâu hay rớt. Như bạn có thể suy luận từ tập dữ liệu trên, những học sinh nào có xếp loại A hoặc B thì đậu và ngược lại thì rớt.

Tên của các tập dữ liệu này sẽ là giá trị của cột thuộc tính mới gọi là 'status'. Trước khi kết hai bảng này lại với nhau, ta sẽ thêm một cột dữ liệu mới để chứa thông tin này.

Sử dụng hàm mutate() để thêm cột thuộc tính mới cho bảng passed. Cột thuộc tính này có tên là status với giá trị là passed (kiểu character string). Tương tự cho cột thuộc tính ở bảng failed.

In [22]:
# mutate() adds new variables and preserves existing; transmute() drops existing variables. 
args(mutate)

function (.data, ...) 
NULL

In [23]:
passed <- passed %>% mutate(status = "passed")
passed

name,class,final,status
Brian,1,B,passed
Roger,2,A,passed
Roger,5,A,passed
Karen,4,A,passed


In [24]:
failed <- failed %>% mutate(status = "failed")
failed

name,class,final,status
Brian,5,C,failed
Sally,1,C,failed
Sally,3,C,failed
Jeff,2,E,failed
Jeff,4,C,failed
Karen,3,C,failed


Ta sử dụng hàm bind_rows() để kết hai bảng này lại

In [25]:
args(bind_rows)

function (..., .id = NULL) 
NULL

In [26]:
bind_rows(passed, failed)

name,class,final,status
Brian,1,B,passed
Roger,2,A,passed
Roger,5,A,passed
Karen,4,A,passed
Brian,5,C,failed
Sally,1,C,failed
Sally,3,C,failed
Jeff,2,E,failed
Jeff,4,C,failed
Karen,3,C,failed


Tất nhiên, chúng ta có thể sắp xếp các dòng dữ liệu theo ý của mình, nhưng điều quan trọng ở đây là mỗi dòng dữ liệu thể hiện một loại đối tượng, mỗi cột dữ liệu là một thuộc tính, và một bảng chỉ chứa một loại đối tượng duy nhất. Như vậy, tập dữ liệu của chúng ta là tidy data.

Chúng ta đã khảo sát khá nhiều ở bài viết này. Bây giờ là lúc tổng hợp mọi thứ để làm việc với dữ liệu thực tế. SAT là một chứng chỉ thông dụng kiểm tra trình độ toán học ở Mĩ gồm: kĩ năng đọc hiểu, toán, và kĩ năng viết luận. Các sinh viên có thể đạt tối đa 800 điểm ở mỗi phần. Tập dữ liệu sau chứa thông tin các sinh viên thực hiện cuộc kiểm tra này. Lấy từ tập dữ liệu 'Total Group Report 2013'. Có thể download tại: http://research.collegeboard.org/programs/sat/data/cb-seniors-2013.

In [27]:
sat

score_range,read_male,read_fem,read_total,math_male,math_fem,math_total,write_male,write_fem,write_total
700-800,40151,38898,79049,74461,46040,120501,31574,39101,70675
600-690,121950,126084,248034,162564,133954,296518,100963,125368,226331
500-590,227141,259553,486694,233141,257678,490819,202326,247239,449565
400-490,242554,296793,539347,204670,288696,493366,262623,302933,565556
300-390,113568,133473,247041,82468,131025,213493,146106,144381,290487
200-290,30728,29154,59882,18788,26562,45350,32500,24933,57433


In [28]:
# 1. select() all columns that do NOT contain the word "total",
# since if we have the male and female data, we can always
# recreate the total count in a separate column, if we want it.
# Hint: Use the contains() function, which you'll
# find detailed in 'Special functions' section of ?select.
#
# 2. gather() all columns EXCEPT score_range, using
# key = part_sex and value = count.
#
# 3. separate() part_sex into two separate variables (columns),
# called "part" and "sex", respectively. You may need to check
# the 'Examples' section of ?separate to remember how the 'into'
# argument should be phrased.
# 4. Use group_by() (from dplyr) to group the data by part and
# sex, in that order.
#
# 5. Use mutate to add two new columns, whose values will be
# automatically computed group-by-group:
#
#   * total = sum(count)
#   * prop = count / total

sat %>%
select(-contains("total")) %>%
gather(part_sex, count, -score_range) %>%
separate(part_sex, c("part", "sex")) %>%
group_by(part, sex) %>%
mutate(total = sum(count),
     prop = count / total
) %>% head

score_range,part,sex,count,total,prop
700-800,read,male,40151,776092,0.05173485
600-690,read,male,121950,776092,0.15713343
500-590,read,male,227141,776092,0.29267278
400-490,read,male,242554,776092,0.31253253
300-390,read,male,113568,776092,0.14633317
200-290,read,male,30728,776092,0.03959324


Ở bài viết này, chúng ta đã học được các dọn dẹp dữ liệu với tidyr và dplyr. Các công cụ này sẽ giúp bạn dành ít thời gian và công sức trong việc lấy và làm sạch dữ liệu để có nhiều thời gian hơn chuẩn bị cho các các bước phân tích kế tiếp.

<strong>Nguồn tham khảo:</strong> <a href="http://swirlstats.com/" target="_blank" rel="noopener">http://swirlstats.com/</a>

<strong>Tham khảo thêm: </strong><a href="http://www.datasciencecentral.com/profiles/blogs/introduction-to-data-quality" target="_blank" rel="noopener">Introduction to data quality</a>